# Classification

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Importation des données

Le fichier "AffinitesVotesONU.txt" est organisé comme indiqué ci-dessous :
- Une ligne renseigne la similarité des votes pour deux pays pendant une année.
- Les états sont codés par les champs "statea" et "stateb". 
- Le champ  "year" donne l'année considérée pour la comparaison des votes.
- Plusieurs indices de comparaison des votes sont fournis, dans la suite on s'intéresse à l'indice de similarité sur 3 niveaux  de votes (oui / non / abstension) :  "agree3un".

Le fichier "StateList.csv" est organisé comme indiqué ci-dessous :
- Une ligne correspond à un pays.
- Le champ "StateAbb" nous donne les abréviations (utile pour les représentations graphiques)
- Le champ "state" est le même code que celui utilisé dans VotingFull
- Le champ "CodeTP" est simplement le numéro de ligne
- Les champs "OPEP" et "UE" renseignent l'année d'entrée dans l'OPEP ou dans l'UE (0 sinon).

Les données étudiées dans ce TP provienne de la plateforme de données [Dataverse](https://dataverse.harvard.edu/dataset.xhtml?persistentId=hdl:1902.1/12379), elles contiennent des indices de similarité entre les votes des pays à l'ONU. 

### Question 1.
Importez les données et affichez les premières lignes de chaque tableau de données. Vous nommerez <code>VotingFull</code> le dataframe contenant les données de "AffinitesVotesONU.txt" et vous nommerez <code>ListCountry</code> le dataframe contenant les données de "StateList.csv"

Affichage des premières lignes de `VotingFull` et `ListCountry`:

# Extraction des similarités de vote pour une periode donnée

In [2]:
# periode étudiée :
annee_debut = 2000
annee_fin  = 2015   

# indice max des indices "state" (.values : syntaxe numpy )
maxstate = max(ListCountry.state.values)
print("indice max de state : " + str(maxstate))

# indice max dans CodeTP : 
maxCodeTP = max(ListCountry.CodeTP.values)
print("indice max de CodeTP : " + str(maxCodeTP))

NameError: name 'ListCountry' is not defined

Une ligne de `VotingFull` nous renseigne sur la dissemblance entre deux pays (indiqués par le codage `state`). Cependant utiliser le codage `state` pour définir les lignes et les colonnes de la matrice de dissemblance pose problème car le code `state` oublie de nombreux entiers, la matrice de dissemblance ainsi construite comporterait de nombreuses lignes et colonnes inutiles. Nous allons donc utiliser le code ad-hoc `CodeTP` qui lui ne saute pas d'entiers.

Le vecteur `Translate` ci-dessous permet de passer du codage `state` au codage `CodeTP`.

In [ ]:
Translate = - np.ones(maxstate+1,dtype = int) 
for numline in range(maxCodeTP):
    Translate[ListCountry.state[numline]] =  ListCountry.CodeTP[numline]

Par exemple le pays de code statea = 40 a pour indice 3 (on part de 0) pour "CodeTP" :

In [ ]:
Translate[40]

### Création de la matrice de dissemblance `D` 

Les numéros de ligne et de colonnes de $D$ correspondent au code "CodeTP". On remplit la matrice de dissemblance en parcourant les lignes de VotingFull et en ne sélectionnant que celles dont l'année appartient à la période étudiée.

La dissemblance d'une année t est définie par :  1 - agree3un(t).

On cumule les dissemblances sur la periode étudiée : 
$$D (paysA,paysB) = \sum_{t= debut} ^{fin}  (1- agree3un(t)) $$ 

On commence par créer une matrice de dissemblance de bonne dimension (remplie de zeros) :

In [ ]:
D = np.zeros((maxCodeTP+1,maxCodeTP+1)) 

### Question 2.
Parcourez les lignes de <code>VotingFull</code> pour rempir la matrice $D$.

Cependant, certaines lignes n'ont pas été remplies. En effet tous les pays ne sont pas renseignés dans la matrice car certains pays n'existent pas ou plus à la date choisie. 
Par exemple : 

In [ ]:
D[50,]

On doit retirer ces pays de la matrice de dissemblance, ce sont les pays dont les lignes ou colonnes ont une somme égale à 0 

### Question 3.
Calculez la matrice $Dextract$ qui correspond à la matrice de dissemblance $D$ dont on a enlevé les lignes ou colonnes qui ont une somme égale à 0.

## Clustering hierarchique avec la librairie Scipy

Le module cluster de la librairie Scipy contient de nombreuses fonctions permettant de mettre en oeuvre les méthodes de classification non supervisée. Nous allons utiliser la fonction hierarchy de ce module, qui offre notamment la possibilité de tracer le dendrogramme, voir http://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html#module-scipy.cluster.hierarchy.

La première étape pour effectuer une classification hierarchique en utilisant consiste à calculer la matrice de distance. Classiquement cette étape est effectuée par la fonction pdist de scipy : http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.distance.pdist.html Cette fonction renvoie une matrice sous forme condensée (en effet la matrice de distance étant symétrique, il n'est pas nécessaire de stocker tous ses coefficients). La seconde étape consiste à appliquer la fonction linkage à la matrice de distance condensée http://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.cluster.hierarchy.linkage.html

Dans le cas présent, nous utiliserons la matrice de dissemblance calculée auparavant. Pour pouvoir lui appliquer la fonction linkage, il nous faut préalablement convertir la matrice de dissemblance sous une "forme condensée" :


In [ ]:
import scipy.spatial.distance as ssd # module scipy pour manipuler des distances 
distArray = ssd.squareform(Dextract)

### Question 4.
Appliquez la fonction <code>linkage</code> du module <code>cluster.hierarchy</code> pour produire un clustering dont vous représenterez le dendrogramme: 
- avec la méthode de plus courte distance (single linkage)
- avec la méthode de Ward (ward linkage)

Lorsqu'on ne connait pas le nombre de classes qu'il pourrait y avoir dans les données, on peut essayer de le "deviner" à partir du dendrogram. Pour cela, il existe plusieurs méthodes, l'une d'entre elle consiste à garder les classes à l'étape $i$ qui maximise la valeur $|h_{i+1} - h_i |$ où $h_i$ est la dissemblance entre les deux classes que l'on a fusionné à l'étape $i$. Sur un dendrogram, les $h_i$ correspondent aux plateaux et comme les $h_i$ sont croissant par rapport à $i$, la technique précedente revient à couper le graphique là où le saut entre deux plateaux consécutifs est le plus grand.

### Question 5.
Déterminez avec la méthode décrite précédemment combien de classes devrait-on garder pour le dendrogram réalisé avec la méthode de Ward. Utilisez la fonction <code>fcluster</code> du module <code>cluster.hierarchy</code> pour retrouver les classes. Pour chaque classe, affichez les noms des pays y appartenant (Attention, lors du calcul de la matrice de dissemblance nous avons enlever des pays qui n'avaient pas voté durant la période étudiée, il faudra penser à les enlever ici aussi). 

### Question 6.
Affichez un diagramme en bâtons avec un bâton qui représente le nombre de pays ayant rejoint l'OPEP (par classes)  et avec un bâton qui représente le nombre de pays ayant rejoint l'UE (par classes). Que remarquez-vous ?